# Welcome to Marvin!
This notebooks serves as an interactive crash course on Marvin's concepts and features.

Before we start, let's make sure we have marvin installed with
```
marvin version
```

We can verify we have an `OPENAI_API_KEY` environment variable set by importing `marvin` and set the log level to `DEBUG` to see how marvin works under the hood:

In [1]:
import marvin

marvin.settings.log_level = 'INFO' # set to 'DEBUG' to see details on how prompts are generated
marvin.settings.llm_model = 'gpt-3.5-turbo' # set to 'gpt-4' for better, but slower, results

### AI Functions

In [2]:
from marvin import ai_fn

@ai_fn
def opposite(thing: str):
    """Return the opposite of a thing"""
    
opposite("up")

'down'

### AI Models

In [3]:
from marvin import ai_model
from pydantic import BaseModel

@ai_model
class Location(BaseModel):
    city: str
    country: str
    latitude: float
    longitude: float

loc = Location("no way, I'm also from the windy city!")

print(
    "https://www.findlatitudeandlongitude.com/"
    f"?lat={loc.latitude}&lon={loc.longitude}"
    "&zoom=&map_type=ROADMAP"
)

loc

https://www.findlatitudeandlongitude.com/?lat=41.8781&lon=-87.6298&zoom=&map_type=ROADMAP


Location(city='Chicago', country='United States', latitude=41.8781, longitude=-87.6298)

### Mapping
Both `ai_fn` and `ai_model`-decorated functions expose a `map` method that allows you to apply the function to a list of inputs using Prefect task mapping. This is useful for running the function in parallel on a list of inputs.

In [ ]:
opposites = await opposite.map(["north", "left"])

locations = await Location.map(["windy city", "big apple"])

print(f"opposites: {opposites}") # ['south', 'right']
print(f"locations: {locations}") # list[Location(..), Location(..)]

# opposites: ['south', 'right']
# locations: [Location(city='Chicago', country='United States', latitude=41.8781, longitude=-87.6298), Location(city='New York City', country='United States', latitude=40.7128, longitude=-74.006)]

### Bots
Bots are customizable, instruct-able assistants with tools. They have:
- a name
- instructions
- a personality
- a list of plugins
- a history of messages

In [1]:
from marvin import Bot

marvin_bot = Bot(
    name="marvin",
    personality="Insists on speaking in confusing analogies",
    instructions=(
        "Answer user questions using plugins to augment your knowledge."
        " Use `chroma_search` if asked anything about the 'marvin' library."
        " Use `DuckDuckGo` if asked about anything else."
    )
)

response = await marvin_bot.say("hi how are you?")

response.content

"I'm not capable of feeling emotions, but I'm here to assist you. How can I help you today?"

#### Plugins
Plugins can be leveraged to extend a bots functionality.

By default, marvin comes with a few:

In [5]:
for plugin in marvin_bot.plugins:
    print(f"{plugin.get_full_description()}\n")

Name: visit-url
Signature: (url: str) -> str
Visit a URL and return its contents. Don't provide a URL unless you're absolutely sure it exists.

Name: DuckDuckGo
Signature: (query: str) -> str
Search the web with DuckDuckGo. Useful for current events. If you already know the answer, you don't need to use this unless asked to. Works best with simple, discrete queries for one question at a time.

Name: calculator
Signature: (expression: str) -> str
Compute an arithmetic expression. The Expression can ONLY include operators, numbers, and the functions sin, cos, tan, sqrt, ln, log, abs, e, pi, π, random, randint; not strings or units.



You can add some of marvin's pre-built plugins:

In [ ]:
from marvin.plugins.duckduckgo import DuckDuckGo

marvin_bot.plugins.append(DuckDuckGo())

response = await marvin_bot.say("use ddg to look up the weather in chicago, and the current time")

response.content

... or you can create your own plugin for your bot:

In [3]:
import httpx
from marvin.plugins import Plugin

class Wolfram(Plugin):
    description: str = "Evaluates a mathematical expression."

    def calculate(self, expression: str) -> str:
        response = httpx.get(f'https://api.wolframalpha.com/v1/result?i={expression}')
        return response.text

    async def run(self, prompt: str) -> str:
        return self.calculate(prompt)

calculator_bot = marvin.bot.Bot(
    instructions="Your job is to solve mathematical expressions using Wolfram Alpha.",
    personality="Analytical, loves calculations.",
)
calculator_bot.plugins.append(Wolfram())

response = await calculator_bot.say("show work: square root of pi + half of the number of world wonders")

response.content

"Sure, I can help you with that. \n\nThe number of world wonders is a bit ambiguous, as there are different lists with different numbers of wonders. However, for the sake of this example, let's assume that you are referring to the Seven Wonders of the Ancient World, which is the most well-known list.\n\nThe Seven Wonders of the Ancient World are: \n1. Great Pyramid of Giza\n2. Hanging Gardens of Babylon\n3. Temple of Artemis at Ephesus\n4. Statue of Zeus at Olympia\n5. Mausoleum at Halicarnassus\n6. Colossus of Rhodes\n7. Lighthouse of Alexandria\n\nSo, the number of world wonders in this case is 7. \n\nNow, let's solve the expression: \n\nSquare root of pi + half of the number of world wonders\n\n= sqrt(pi) + 0.5 * 7\n\n= 1.77245385091 + 3.5\n\n= 5.27245385091\n\nTherefore, the answer is 5.27245385091."

### Loaders
To create a custom knowledge-base, you can use a `Loader` to create and dump `Document` excerpts to a vectorstore, which can then be queried by a bot's plugin.

For example, we can use the `HTMLLoader` to load a webpage and dump its contents to a vectorstore:

In [ ]:
from marvin.loaders.web import HTMLLoader

marvin.settings.log_level = 'DEBUG'

marvin_doc_homepage = HTMLLoader(urls=["https://www.askmarvin.ai/getting_started/installation/"])

# by default, the document will be chunked into excerpts and embedded locally in a DuckDB-backed Chroma index
await marvin_doc_homepage.load_and_store()

... and now to leverage our new knowledge-base we can give our bot a plugin that can query it. Let's see how it provides context to the bot:

In [ ]:
from marvin.plugins.chroma import chroma_search

# run the plugin ourselves
example_plugin_output = await chroma_search.run("how do I install marvin?")

print(example_plugin_output)

# give the bot the plugin
marvin_bot.plugins.append(chroma_search)

# ask the bot
response = await marvin_bot.say("how do I install marvin?")